<a href="https://colab.research.google.com/github/jaysalomon/Bio-Spin/blob/main/Reservoir_Computing_Frequency_Classification_Code_(CPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

# --- Added imports for Classification Task ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression # Or RidgeClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
# May need: !pip install scikit-learn

# --- Assume Magnetosome and Simulation classes are defined here ---
# <<< PASTE the Magnetosome and Simulation class definitions here >>>
# --- Physical Constants ---
GAMMA_LL = 2.2128e5
ALPHA = 0.05
MS = 4.8e5
KB = 1.380649e-23
MU0 = 4 * np.pi * 1e-7

class Magnetosome:
    """ Represents a single magnetosome nanoparticle. """
    def __init__(self, position, initial_M_direction, volume, Ku, easy_axis=[0, 0, 1]):
        self.pos = np.array(position, dtype=float)
        initial_M_direction = np.array(initial_M_direction, dtype=float)
        norm = np.linalg.norm(initial_M_direction)
        if norm < 1e-9: raise ValueError("Initial magnetization direction cannot be a zero vector.")
        self.M = (initial_M_direction / norm) * MS
        self.V = float(volume)
        self.Ku = float(Ku)
        self.Ms = MS
        self.easy_axis = np.array(easy_axis, dtype=float)
        norm_ea = np.linalg.norm(self.easy_axis)
        if norm_ea < 1e-9: raise ValueError("Easy axis cannot be a zero vector.")
        self.easy_axis /= norm_ea

    def anisotropy_field(self):
        M_norm = np.linalg.norm(self.M)
        if M_norm < 1e-9: return np.zeros(3)
        M_unit = self.M / M_norm
        M_physical = M_unit * self.Ms
        M_dot_easy_axis = np.dot(M_physical, self.easy_axis)
        H_anis_magnitude = (2 * self.Ku) / (MU0 * self.Ms)
        H_anis = H_anis_magnitude * (M_dot_easy_axis / self.Ms) * self.easy_axis
        return H_anis

    def normalize_M(self):
         norm_M = np.linalg.norm(self.M)
         if norm_M > 1e-9: self.M = (self.M / norm_M) * self.Ms

class Simulation:
    """ Manages the simulation environment. """
    def __init__(self, magnetosomes, H_ext_func, temperature=0, approx_dt=1e-12):
        self.magnetosomes = magnetosomes
        self.H_ext_func = H_ext_func
        self.temperature = temperature
        self.approx_dt = approx_dt

    def calculate_dipolar_field(self, target_index, current_M_vectors):
        H_dip = np.zeros(3)
        target_magnetosome = self.magnetosomes[target_index]
        for i, other_magnetosome in enumerate(self.magnetosomes):
            if i == target_index: continue
            r_vec = target_magnetosome.pos - other_magnetosome.pos
            r_dist = np.linalg.norm(r_vec)
            if r_dist < 1e-12: continue
            r_hat = r_vec / r_dist
            M_j = current_M_vectors[i]
            m_j = M_j * other_magnetosome.V
            term1 = 3 * np.dot(m_j, r_hat) * r_hat; term2 = m_j
            H_dip += (1 / (4 * np.pi * r_dist**3)) * (term1 - term2)
        return H_dip

    def calculate_thermal_field(self, magnetosome):
        if self.temperature <= 0 or self.approx_dt <= 0: return np.zeros(3)
        factor = np.sqrt((2*ALPHA*KB*self.temperature)/(GAMMA_LL*MU0*magnetosome.Ms*magnetosome.V*self.approx_dt))
        return factor * np.random.normal(0.0, 1.0, 3)

    def calculate_Heff(self, index, current_M_vectors, t):
        magnetosome = self.magnetosomes[index]
        H_ext_t = self.H_ext_func(t)
        original_M = magnetosome.M.copy()
        magnetosome.M = current_M_vectors[index]
        H_anis = magnetosome.anisotropy_field()
        magnetosome.M = original_M
        H_dip = self.calculate_dipolar_field(index, current_M_vectors)
        H_th = self.calculate_thermal_field(magnetosome)
        return H_ext_t + H_anis + H_dip + H_th

    def llg_equation(self, t, M_flat):
        dM_dt_flat = np.zeros_like(M_flat)
        num_magnetosomes = len(self.magnetosomes)
        current_M_vectors = {i: M_flat[i*3:(i+1)*3] for i in range(num_magnetosomes)}
        for i in range(num_magnetosomes):
            M = current_M_vectors[i]; Ms_i = self.magnetosomes[i].Ms
            if Ms_i < 1e-9 or np.linalg.norm(M) < 1e-9: continue
            H_eff = self.calculate_Heff(i, current_M_vectors, t)
            pre1 = -GAMMA_LL / (1 + ALPHA**2)
            safe_Ms = Ms_i if Ms_i > 1e-12 else 1.0
            pre2 = pre1 * ALPHA / safe_Ms
            cross1 = np.cross(M, H_eff); cross2 = np.cross(M, cross1)
            dM_dt = pre1 * cross1 + pre2 * cross2
            dM_dt_flat[i*3:(i+1)*3] = dM_dt
        return dM_dt_flat

    def run(self, t_span, dt_max, t_eval=None):
        self.approx_dt = dt_max
        initial_M_flat = np.concatenate([m.M for m in self.magnetosomes])
        print(f"Running simulation from t={t_span[0]} to t={t_span[1]}...")
        start_sim_time = time.time()
        sol = solve_ivp(fun=self.llg_equation, t_span=t_span, y0=initial_M_flat,
                        method='RK45', max_step=dt_max, t_eval=t_eval)
        end_sim_time = time.time()
        print(f"Simulation finished in {end_sim_time - start_sim_time:.2f} seconds.")
        if sol.status != 0: print(f"Warning: Solver status {sol.status}: {sol.message}")
        # Update final state (optional)
        final_M_flat = sol.y[:, -1]
        for i, m in enumerate(self.magnetosomes):
            m.M = final_M_flat[i*3:(i+1)*3]; m.normalize_M()
        return sol

# <<< END of Simulation class definition >>>


# ==========================================================
# --- Reservoir Setup (Same as before) ---
# ==========================================================
num_magnetosomes_reservoir = 20
reservoir_size = 300e-9
magnetosome_diameter = 45e-9
magnetosome_radius = magnetosome_diameter / 2
magnetosome_volume = (4/3) * np.pi * magnetosome_radius**3
magnetosome_Ku = 1.1e4
easy_axis_direction = [0, 0, 1]

# --- Create Reservoir Structure (Function for reusability) ---
def create_reservoir(n_magnetosomes, size, vol, Ku, easy_axis):
    magnetosomes = []
    positions = []
    print(f"Generating {n_magnetosomes} random magnetosome positions...")
    attempts = 0; max_attempts = n_magnetosomes * 100
    while len(magnetosomes) < n_magnetosomes and attempts < max_attempts:
        pos = (np.random.rand(3) - 0.5) * size
        # Optional proximity check could be added here
        random_tilt = (np.random.rand(3) - 0.5) * 0.1
        initial_M = np.array(easy_axis) + random_tilt
        m = Magnetosome(position=pos, initial_M_direction=initial_M, volume=vol, Ku=Ku, easy_axis=easy_axis)
        magnetosomes.append(m)
        positions.append(pos)
        attempts += 1
    if len(magnetosomes) < n_magnetosomes: print(f"Warning: Only generated {len(magnetosomes)}.")
    print(f"Generated reservoir with {len(magnetosomes)} magnetosomes.")
    return magnetosomes

# Create the reservoir structure once
reservoir_structure = create_reservoir(num_magnetosomes_reservoir, reservoir_size,
                                     magnetosome_volume, magnetosome_Ku, easy_axis_direction)

# --- Simulation Parameters ---
input_amplitude = 8e3 # A/m (~10 mT)
freq1 = 1.0e9 # Hz (1 GHz) - Class 0
freq2 = 1.5e9 # Hz (1.5 GHz) - Class 1

simulation_temperature = 0 # No thermal noise

t_start = 0
num_cycles_max_freq = 8 # Simulate for enough cycles of the higher frequency
t_end = num_cycles_max_freq / max(freq1, freq2) # Ensure comparable duration
dt_max = 1e-12
num_time_points = 801 # More points might be needed
t_eval_points = np.linspace(t_start, t_end, num_time_points)

# --- Define Function to Run Simulation for a Given Frequency ---
def run_simulation_for_freq(freq, structure):
    # Reset initial M state for the structure before each run
    for m in structure:
         random_tilt = (np.random.rand(3) - 0.5) * 0.1
         initial_M = np.array(easy_axis_direction) + random_tilt
         norm = np.linalg.norm(initial_M)
         m.M = (initial_M / norm) * MS if norm > 1e-9 else np.array(easy_axis_direction) * MS

    # Define H_ext function for this frequency
    def H_ext_func(t):
        signal = np.array([0, input_amplitude * np.sin(2 * np.pi * freq * t), 0])
        return signal

    sim = Simulation(magnetosomes=structure, H_ext_func=H_ext_func, temperature=simulation_temperature)
    solution = sim.run(t_span=[t_start, t_end], dt_max=dt_max, t_eval=t_eval_points)
    return solution.y, solution.t # Return states and times

# ==========================================================
# --- 1. Generate Data for Two Classes ---
# ==========================================================
print(f"\n--- Generating data for Frequency 1 ({freq1/1e9:.1f} GHz) ---")
reservoir_states1, times1 = run_simulation_for_freq(freq1, reservoir_structure)

print(f"\n--- Generating data for Frequency 2 ({freq2/1e9:.1f} GHz) ---")
reservoir_states2, times2 = run_simulation_for_freq(freq2, reservoir_structure)

# Ensure times are consistent (should be if t_eval was used correctly)
if not np.allclose(times1, times2):
    print("Warning: Time points differ between runs. Check t_eval.")
    # Consider interpolating or using only common time points if necessary
times = times1 # Use times from first run

# ==========================================================
# --- 2. Prepare Data for Classifier ---
# ==========================================================
# Features X: Combine states, transpose for sklearn (samples, features)
# Shape (T, 3N) for each run -> Stack vertically
X1 = reservoir_states1.T
X2 = reservoir_states2.T
X = np.vstack([X1, X2]) # Shape (2*T, 3N)

# Labels y: 0 for freq1, 1 for freq2
y1 = np.zeros(X1.shape[0], dtype=int)
y2 = np.ones(X2.shape[0], dtype=int)
y = np.concatenate([y1, y2]) # Shape (2*T,)

print(f"\nCombined feature matrix X shape: {X.shape}")
print(f"Combined label vector y shape: {y.shape}")

# Optional: Remove initial transient phase (washout)
washout_time = 2 / min(freq1, freq2) # Example: washout for 2 cycles of slowest freq
washout_index = np.searchsorted(times, washout_time)

if washout_index > 0 and washout_index < X1.shape[0]:
    print(f"Applying washout: removing first {washout_index} time steps from each run.")
    # Apply washout to combined data
    indices_to_keep = np.concatenate([
        np.arange(washout_index, X1.shape[0]),             # Indices for run 1 after washout
        np.arange(X1.shape[0] + washout_index, X.shape[0]) # Indices for run 2 after washout
    ])
    X = X[indices_to_keep, :]
    y = y[indices_to_keep]
    print(f"Shape after washout - X: {X.shape}, y: {y.shape}")
else:
    print("Washout not applied or covers entire dataset.")


# ==========================================================
# --- 3. Train/Test Split ---
# ==========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

# ==========================================================
# --- 4. Feature Scaling ---
# ==========================================================
print("Scaling features using StandardScaler...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) # Use transform only on test set

# ==========================================================
# --- 5. Train Classifier ---
# ==========================================================
# Use Logistic Regression
classifier = LogisticRegression(solver='liblinear', random_state=42) # liblinear often good for smaller datasets

print("Training Logistic Regression classifier...")
start_train_time = time.time()
classifier.fit(X_train_scaled, y_train)
end_train_time = time.time()
print(f"Training finished in {end_train_time - start_train_time:.3f} seconds.")

# ==========================================================
# --- 6. Evaluate Classifier ---
# ==========================================================
print("\n--- Evaluating Classifier Performance ---")
y_pred = classifier.predict(X_test_scaled)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy: {accuracy:.4f}")

# Print Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=[f'{freq1/1e9:.1f} GHz', f'{freq2/1e9:.1f} GHz']))

# Plot Confusion Matrix
print("\nPlotting Confusion Matrix...")
fig, ax = plt.subplots(figsize=(6, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred,
    ax=ax,
    display_labels=[f'{freq1/1e9:.1f} GHz', f'{freq2/1e9:.1f} GHz'],
    cmap=plt.cm.Blues
)
ax.set_title('Confusion Matrix for Frequency Classification')
plt.tight_layout()
plt.show()

print("\nFrequency classification task complete.")
# High accuracy indicates the reservoir states are linearly separable
# based on the input frequency.

Generating 20 random magnetosome positions...
Generated reservoir with 20 magnetosomes.

--- Generating data for Frequency 1 (1.0 GHz) ---
Running simulation from t=0 to t=5.333333333333333e-09...
